# Unigram Tokenization

## 1. Implementing Unigram

Now let’s implement everything we’ve seen so far in code. Like with BPE and WordPiece, this is not an efficient implementation of the Unigram algorithm (quite the opposite), but it should help you understand it a bit better.



We will use the same corpus as before as an example:



In [1]:
corpus = [
    "This is the Hugging Face Course.",
    "This chapter is about tokenization.",
    "This section shows several tokenizer algorithms.",
    "Hopefully, you will be able to understand how they are trained and generate tokens."
]

This time, we will use `xlnet-base-cased` as our model:



In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("xlnet-base-cased")

Like for BPE and WordPiece, we begin by counting the number of occurrences of each word in the corpus:



In [3]:
from collections import defaultdict

word_freqs = defaultdict(int)
for text in corpus:
    pre_tokenized_result = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    pre_tokenized_text = [word for word, offset in pre_tokenized_result]
    for word in pre_tokenized_text:
        word_freqs[word] += 1
word_freqs

defaultdict(int,
            {'▁This': 3,
             '▁is': 2,
             '▁the': 1,
             '▁Hugging': 1,
             '▁Face': 1,
             '▁Course.': 1,
             '▁chapter': 1,
             '▁about': 1,
             '▁tokenization.': 1,
             '▁section': 1,
             '▁shows': 1,
             '▁several': 1,
             '▁tokenizer': 1,
             '▁algorithms.': 1,
             '▁Hopefully,': 1,
             '▁you': 1,
             '▁will': 1,
             '▁be': 1,
             '▁able': 1,
             '▁to': 1,
             '▁understand': 1,
             '▁how': 1,
             '▁they': 1,
             '▁are': 1,
             '▁trained': 1,
             '▁and': 1,
             '▁generate': 1,
             '▁tokens.': 1})

Then, we need to initialize our vocabulary to something larger than the vocab size we will want at the end. We have to include all the basic characters (otherwise we won’t be able to tokenize every word), but for the bigger substrings we’ll only keep the most common ones, so we sort them by frequency:



In [4]:
char_freqs = defaultdict(int)
subword_freqs = defaultdict(int)
for word, freq in word_freqs.items():
    for i in range(len(word)):
        char_freqs[word[i]] += freq
        for j in range(i + 2, len(word) + 1):
            subword_freqs[word[i: j]] += freq

# Sort subword by frequency
sorted_subwords = sorted(subword_freqs.items(), key=lambda x: x[1], reverse=True)
sorted_subwords

[('▁t', 7),
 ('is', 5),
 ('er', 5),
 ('▁a', 5),
 ('▁to', 4),
 ('to', 4),
 ('en', 4),
 ('▁T', 3),
 ('▁Th', 3),
 ('▁Thi', 3),
 ('▁This', 3),
 ('Th', 3),
 ('Thi', 3),
 ('This', 3),
 ('hi', 3),
 ('his', 3),
 ('th', 3),
 ('ou', 3),
 ('se', 3),
 ('▁tok', 3),
 ('▁toke', 3),
 ('▁token', 3),
 ('tok', 3),
 ('toke', 3),
 ('token', 3),
 ('ok', 3),
 ('oke', 3),
 ('oken', 3),
 ('ke', 3),
 ('ken', 3),
 ('▁s', 3),
 ('ra', 3),
 ('nd', 3),
 ('▁i', 2),
 ('▁is', 2),
 ('▁th', 2),
 ('▁the', 2),
 ('the', 2),
 ('he', 2),
 ('▁H', 2),
 ('in', 2),
 ('rs', 2),
 ('te', 2),
 ('▁ab', 2),
 ('ab', 2),
 ('▁tokeni', 2),
 ('▁tokeniz', 2),
 ('tokeni', 2),
 ('tokeniz', 2),
 ('okeni', 2),
 ('okeniz', 2),
 ('keni', 2),
 ('keniz', 2),
 ('eni', 2),
 ('eniz', 2),
 ('ni', 2),
 ('niz', 2),
 ('iz', 2),
 ('at', 2),
 ('ti', 2),
 ('tio', 2),
 ('tion', 2),
 ('io', 2),
 ('ion', 2),
 ('on', 2),
 ('▁se', 2),
 ('ho', 2),
 ('how', 2),
 ('ow', 2),
 ('era', 2),
 ('al', 2),
 ('s.', 2),
 ('ll', 2),
 ('an', 2),
 ('and', 2),
 ('ne', 2),
 ('▁Hu',

In [5]:
len(sorted_subwords)

564

We group the characters with the best subwords to arrive at an initial vocabulary of size 300:



In [6]:
token_freqs = list(char_freqs.items()) + sorted_subwords[: 300 - len(char_freqs)]
token_freqs = {token: freq for token, freq in token_freqs}

💡 SentencePiece uses a more efficient algorithm called Enhanced Suffix Array (ESA) to create the initial vocabulary.



Next, we compute the sum of all frequencies, to convert the frequencies into probabilities. For our model we will store the logarithms of the probabilities, because it’s more numerically stable to add logarithms than to multiply small numbers, and this will simplify the computation of the loss of the model:



In [7]:
from math import log

total_sum = sum([freq for token, freq in token_freqs.items()])
model = {token: -log(freq / total_sum) for token, freq in token_freqs.items()}

Now the main function is the one that tokenizes words using the Viterbi algorithm. As we saw before, that algorithm computes the best segmentation of each substring of the word, which we will store in a variable named `best_segmentations`. We will store one dictionary per position in the word (from 0 to its total length), with two keys: the index of the start of the last token in the best segmentation, and the score of the best segmentation. With the index of the start of the last token, we will be able to retrieve the full segmentation once the list is completely populated.



Populating the list is done with just two loops: the main loop goes over each start position, and the second loop tries all substrings beginning at that start position. If the substring is in the vocabulary, we have a new segmentation of the word up until that end position, which we compare to what is in `best_segmentations`.



Once the main loop is finished, we just start from the end and hop from one start position to the next, recording the tokens as we go, until we reach the start of the word:



In [9]:
def encode_word(word, model):
    best_segmentations = [{"start": 0, "score": 1}] + [
        {"start": None, "score": None} for _ in range(len(word))
    ]
    for start_idx in range(len(word)):
        best_score_at_start = best_segmentations[start_idx]["score"]
        for end_idx in range(start_idx + 1, len(word) + 1):
            token = word[start_idx: end_idx]
            if token in model and best_score_at_start is not None:
                score = model[token] + best_score_at_start
                if (
                    best_segmentations[end_idx]["score"] is None
                    or best_segmentations[end_idx]["score"] > score
                ):
                    best_segmentations[end_idx] = {"start": start_idx, "score": score}

    segmentation = best_segmentations[-1]
    if segmentation["score"] is None:
        return ["<unk>"], None
    score = segmentation["score"]
    start = segmentation["start"]
    end = len(word)
    tokens = []
    while start != 0:
        tokens.insert(0, word[start: end])
        next_start = best_segmentations[start]["start"]
        end = start
        start = next_start
    tokens.insert(0, word[start: end])
    return tokens, score

We can already try our initial model on some words:



In [10]:
print(encode_word("Hopefully", model))

(['H', 'o', 'p', 'e', 'f', 'u', 'll', 'y'], 41.5157494601402)


In [12]:
print(encode_word("beautiful", model))

(['b', 'e', 'a', 'ut', 'i', 'f', 'u', 'l'], 39.164374202976724)


Now it’s easy to compute the loss of the model on the corpus!



In [13]:
def compute_loss(model):
    loss = 0
    for word, freq in word_freqs.items():
        _, word_loss = encode_word(word, model)
        loss += word_loss * freq
    return loss

We can check it works on the model we have:



In [14]:
compute_loss(model)


413.10377642940875

Computing the scores for each token is not very hard either; we just have to compute the loss for the models obtained by deleting each token:



In [15]:
import copy

def compute_scores(model):
    scores = {}
    model_loss = compute_loss(model)
    for token, score in model.items():
        # We always keep token of length 1
        if len(token) == 1:
            continue
        model_without_token = copy.deepcopy(model)
        _ = model_without_token.pop(token)
        scores[token] = compute_loss(model_without_token) - compute_loss(model)
    return scores

We can try it on a given token:



In [16]:
scores = compute_scores(model)
print(scores["ll"])
print(scores["his"])

6.376412403623874
0.0


With all of this in place, the last thing we need to do is add the special tokens used by the model to the vocabulary, then loop until we have pruned enough tokens from the vocabulary to reach our desired size:



In [18]:
percent_to_remove = 0.1
while len(model) > 100:
    scores = compute_scores(model)
    sorted_scores = sorted(scores.items(), key=lambda x: x[1])
    for i in range(int(len(model) * percent_to_remove)):
        _ = token_freqs.pop(sorted_scores[i][0])
    total_sum = sum([freq for token, freq in token_freqs.items()])
    model = {token: -log(freq / total_sum) for token, freq in token_freqs.items()}

Then, to tokenize some text, we just need to apply the pre-tokenization and then use our encode_word() function:



In [23]:
def tokenize(text, model):
    pre_tokenized_result = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    pre_tokenized_text = [word for word, offset in pre_tokenized_result]
    encoded_word = [encode_word(word, model)[0] for word in pre_tokenized_text]
    return sum(encoded_word, [])

In [24]:
tokenize("This is the Hugging Face course.", model)

['▁This',
 '▁is',
 '▁the',
 '▁Hugging',
 '▁Face',
 '▁',
 'c',
 'ou',
 'r',
 's',
 'e',
 '.']

The XLNetTokenizer uses SentencePiece which is why the `"_"` character is included. To decode with SentencePiece, concatenate all the tokens and replace `"_"` with a space.

